<a href="https://colab.research.google.com/github/enzocatorano/deep_learning_practicas/blob/master/3_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### **Ejercicio 1**: Inicialización de pesos

En este ejercicio, vamos a probar distintas inicializaciones de pesos para la red `IrisMLP`. Para esto, modificar el constructor de la red de manera que se pueda pasar el método de inicialización como argumento. Pueden definir un método privado `_init_weights` por ejemplo y llamarlo en el constructor (el prefijo `_` es la convención en Python para métodos privados).

a) Probemos inicializar la red `IrisMLP` con todos ceros. Observar qué ocurre con la evolución de los pesos dentro de una misma capa (para lo cual puede ser útil Tensorboard), y con el desempeño final de la red.

b) Implementar la inicialización Xavier, He y con una distribución normal.

Opcionalmente, pueden repetir estos incisos para `NetCNN`.


In [ ]:
import torch
torch.cuda.is_available()
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.tensorboard import SummaryWriter
import time
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from torch.optim import Optimizer
import matplotlib.pyplot as plt
import os
import pandas as pd
from typing import Tuple
from torch.nn import functional as F

class PerceptronMulticapa (nn.Module):

  #############################################################################################################
  def __init__ (self, n_entrada, n_oculta, n_salida, f_act = 'relu',
                semilla = None, metodo_init_pesos = None, batch_norm = False,
                dropout_val = 0.0):
    super(PerceptronMulticapa, self).__init__()
    if semilla is not None:
      torch.manual_seed(semilla)
      self.semilla = semilla
    self.n_entrada = n_entrada
    self.n_oculta = n_oculta
    self.n_salida = n_salida
    if f_act == 'relu':
      self.f_act = nn.ReLU()
    elif f_act == 'sigmoid':
      self.f_act = nn.Sigmoid()
    self.fc1 = torch.nn.Linear(n_entrada, n_oculta, dtype = torch.float32)
    self.fc2 = torch.nn.Linear(n_oculta, n_salida, dtype = torch.float32)
    if metodo_init_pesos is not None:
      self._init_pesos(metodo_init_pesos)
    # necesito agregar implementacion para el escenario en que se use
    # normalizacion por lotes
    def mantener (valor):
      return valor
    if batch_norm:
      self.bn1 = nn.BatchNorm1d(n_oculta)
      self.bn2 = nn.BatchNorm1d(n_salida)
    else: # si no hay que hacer BN, las guarda como funciones que no hacen nada
      self.bn1 = mantener
      self.bn2 = mantener
    # agrego la posibilidad de implementar dropout
    if dropout_val > 0:
      self.dropout1 = nn.Dropout(dropout_val)
      self.dropout2 = nn.Dropout(dropout_val)
    else:
      self.dropout1 = mantener
      self.dropout2 = mantener

  ###################
  def __str__ (self):
    return f"MLP con {self.n_entrada} entradas, {self.n_oculta} ocultas y {self.n_salida} salidas. Funcion de activación {self.f_act}."

  ##############################
  def forward (self, x_entrada):
    if len(x_entrada.shape) == 1:
      x_entrada = x_entrada.unsqueeze(0)
    if x_entrada.shape[1] != self.n_entrada:
      raise ValueError(f"Dimensión incorrecta: se esperaban {self.n_entrada} entradas por dato.")
    y1 = self.fc1(x_entrada)
    z1 = self.dropout1(self.bn1(self.f_act(y1)))
    y2 = self.fc2(z1)
    z2 = self.dropout2(self.bn2(self.f_act(y2)))
    return z2

  ##########################################
  def _init_pesos (self, metodo_init_pesos):
    '''
    Funcion de inicializacion de pesos, espera un metodo de inicializacion como argumento
    del tipo torch.nn.init.Module. Como ser:
    - torch.nn.init.zeros_
    - torch.nn.init.xavier_uniform_
    - torch.nn.init.xavier_normal_
    - torch.nn.init.kaiming_uniform_
    - torch.nn.init.kaiming_normal_
    '''
    for capa in [self.fc1, self.fc2]:
      metodo_init_pesos(capa.weight)
      if capa.bias is not None:
        capa.bias.data.fill_(0) # pone ceros para los sesgos, lo que es comun

##########################################################################################################################################
##########################################################################################################################################
##########################################################################################################################################

class NetCNN(nn.Module):

  ############################################################################
  def __init__ (self, f1 = 5, s1 = 1, p1 = 0, f2 = 5, s2 = 1, p2 = 0,
                semilla = None, metodo_init_pesos = None, batch_norm = False,
                dropout_val = 0.0):
    '''
    Hay parametros no manejables, pues, se corresponden a aquellos dados por consigna.
    Como ser el tipo y tamaño del pooling, la cantidad de features map antes y despues
    de cada capa convolucional, las neuronas de cada capa MLP, y las funciones de activacion.
    '''
    super(NetCNN, self).__init__()
    if semilla != None:
      torch.manual_seed(semilla)
      self.semilla = semilla
    self.fm0 = 1
    self.fm1 = 6
    self.fm2 = 16
    self.relu = nn.ReLU()
    self.conv1 = nn.Conv2d(self.fm0, self.fm1, f1, s1, p1)
    self.pool1 = nn.MaxPool2d(2, 2) # este es el reductor de dimensiones de los features maps
    self.conv2 = nn.Conv2d(self.fm1, self.fm2, f2, s2, p2)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(self.fm2 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    if metodo_init_pesos is not None:
      self._init_pesos(metodo_init_pesos)
    def mantener (valor):
      return valor
    if batch_norm:
      self.cbn1 = nn.BatchNorm2d(self.fm1)
      self.cbn2 = nn.BatchNorm2d(self.fm2)
      self.lbn1 = nn.BatchNorm1d(120)
      self.lbn2 = nn.BatchNorm1d(84)
    else:
      self.cbn1 = mantener
      self.cbn2 = mantener
      self.lbn1 = mantener
      self.lbn2 = mantener
    if dropout_val > 0:
      self.dropout1 = nn.Dropout(dropout_val)
      self.dropout2 = nn.Dropout(dropout_val)
    else:
      self.dropout1 = mantener
      self.dropout2 = mantener

  ##################
  def __str__(self):
    desc = "NetCNN con arquitectura:\n"
    desc += f"- Conv1: entrada {self.fm0} → {self.fm1} filtros\n"
    desc += f"- Conv2: entrada {self.fm1} → {self.fm2} filtros\n"
    desc += f"- FC1: {self.fm2 * 4 * 4} → 120\n"
    desc += f"- FC2: 120 → 84\n"
    desc += f"- FC3: 84 → 10 (salidas)\n"
    desc += f"Activación: ReLU\n"
    desc += f"BatchNorm: {'sí' if hasattr(self, 'cbn1') and not callable(self.bn1) else 'no'}"
    return desc

  ######################
  def forward (self, x):
    h = self.conv1(x)
    h = self.relu(h)
    h = self.cbn1(h)
    h = self.pool1(h)
    h = self.conv2(h)
    h = self.relu(h)
    h = self.cbn2(h)
    h = self.pool2(h)
    h = h.view(-1, self.fm2 * 4 * 4) # aca se aplanan todos los features maps sobre un solo vector
    h = self.fc1(h)
    h = self.relu(h)
    h = self.lbn1(h)
    h = self.dropout1(h)
    h = self.fc2(h)
    h = self.relu(h)
    h = self.lbn2(h)
    h = self.dropout2(h)
    y = self.fc3(h)
    return y # 10 elementos

  ##########################################
  def _init_pesos (self, metodo_init_pesos):
    '''
    Funcion de inicializacion de pesos, espera un metodo de inicializacion como argumento
    del tipo torch.nn.init.Module. Como ser:
    - torch.nn.init.zeros_
    - torch.nn.init.xavier_uniform_
    - torch.nn.init.xavier_normal_
    - torch.nn.init.kaiming_uniform_
    - torch.nn.init.kaiming_normal_
    '''
    for capa in [self.conv1, self.conv2, self.fc1, self.fc2, self.fc3]:
      metodo_init_pesos(capa.weight)
      if capa.bias is not None:
        capa.bias.data.fill_(0) # pone ceros para los sesgos, lo que es comun

##########################################################################################################################################
##########################################################################################################################################
##########################################################################################################################################

class Entrenador:
    def __init__(
        self,
        modelo: nn.Module,
        cargador_entrenamiento: DataLoader,
        optimizador: Optimizer,
        func_perdida: nn.Module,
        cargador_validacion: DataLoader = None,
        device: str | None = None,
        parada_temprana: int | None = 10,
        regularizador: tuple[int, float] = None
    ):
        # Configuración de dispositivo
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
            print(f"Usando dispositivo: {device.upper()}")
        self.device = device
        self.modelo = modelo.to(self.device)
        self.cargador_entrenamiento = cargador_entrenamiento
        self.cargador_validacion = cargador_validacion
        self.optimizador = optimizador
        self.func_perdida = func_perdida
        self.parada_temprana = parada_temprana

        # Bandera para aplicar softmax si se usa MSELoss
        self.usar_softmax = isinstance(self.func_perdida, nn.MSELoss)

        # Validación de formato de regularizador
        if regularizador is not None:
            if (
                not isinstance(regularizador, tuple)
                or len(regularizador) != 2
                or not isinstance(regularizador[0], int)
                or not isinstance(regularizador[1], float)
            ):
                raise ValueError("El regularizador debe ser de formato tuple[int, float].")
            if regularizador[0] not in [1, 2]:
                raise ValueError("El primer elemento del regularizador debe ser 1 o 2.")
            self.regularizador = regularizador
        else:
            self.regularizador = None

        # Construcción de la función de pérdida total con regularización
        if self.regularizador:
            def func_perdida_total(salida, salida_esperada):
                perdida = self.func_perdida(salida, salida_esperada)
                reg = 0.0
                for nombre, param in self.modelo.named_parameters():
                    if "bias" not in nombre and param.requires_grad:
                        reg += torch.norm(param, p=self.regularizador[0])
                return perdida + self.regularizador[1] * reg
            self.func_perdida_total = func_perdida_total
        else:
            self.func_perdida_total = self.func_perdida

        # Historial de métricas
        self.perdida_entrenamiento = []
        self.perdida_validacion = []
        self.precision_validacion = []
        self.mejor_perdida_validacion = float('inf')
        self.mejor_modelo = None
        self.epocas_sin_mejora = 0

    def ajustar(
        self,
        epocas: int,
        imprimir_perdida=False,
        plotear_perdida=True,
        escritor=None
    ):
        # Chequeo inicial de formato de etiquetas
        lote_ejemplo = next(iter(self.cargador_entrenamiento))
        _, etiquetas_ejemplo = lote_ejemplo
        if isinstance(self.func_perdida, nn.CrossEntropyLoss) and etiquetas_ejemplo.ndim != 1:
            raise ValueError("CrossEntropyLoss espera etiquetas como índices enteros (dim = 1).")
        if isinstance(self.func_perdida, nn.MSELoss) and not (
            etiquetas_ejemplo.ndim == 2 and etiquetas_ejemplo.size(1) > 1
        ):
            raise ValueError("MSELoss espera etiquetas en formato one-hot (batch_size, num_classes).")

        # Bucle de entrenamiento
        for t in range(epocas):
            self.modelo.train()
            perdida_epoca = 0.0

            for x_entrada, salida_esperada in self.cargador_entrenamiento:
                x_entrada = x_entrada.to(self.device)
                salida_esperada = salida_esperada.to(self.device)

                # Forward + softmax opcional
                salida = self.modelo(x_entrada)
                if self.usar_softmax:
                    salida = F.softmax(salida, dim=1)

                # Cálculo de pérdida y backward
                perdida = self.func_perdida_total(salida, salida_esperada)
                self.optimizador.zero_grad()
                perdida.backward()
                self.optimizador.step()

                perdida_epoca += perdida.item()

            perdida_prom = perdida_epoca / len(self.cargador_entrenamiento)
            if escritor:
                escritor.add_scalar("Loss/train", perdida_prom, t)
                # Logueo de pesos, gradientes y BatchNorm
                for nombre, param in self.modelo.named_parameters():
                    escritor.add_histogram(f"Weights/{nombre}", param, t)
                    if param.grad is not None:
                        escritor.add_histogram(f"Gradients/{nombre}", param.grad, t)
                    if 'bn' in nombre:
                        escritor.add_histogram(f"BatchNorm/{nombre}", param, t)
                for nombre, buffer in self.modelo.state_dict().items():
                    if 'running_mean' in nombre or 'running_var' in nombre:
                        escritor.add_histogram(f"BatchNorm/{nombre}", buffer, t)
            else:
                self.perdida_entrenamiento.append(perdida_prom)

            if imprimir_perdida:
                print(f"Época {t+1}/{epocas} — Pérdida train: {perdida_prom:.4f}")

            # Validación y parada temprana
            if self.cargador_validacion:
                p_val, acc_val = self.validar(imprimir_perdida)
                self.perdida_validacion.append(p_val)
                self.precision_validacion.append(acc_val)

                if escritor:
                    escritor.add_scalar("Loss/val", p_val, t)
                    escritor.add_scalar("Accuracy/val", acc_val, t)

                if p_val < self.mejor_perdida_validacion:
                    self.mejor_perdida_validacion = p_val
                    self.guardar_checkpoint()
                    self.epocas_sin_mejora = 0
                else:
                    self.epocas_sin_mejora += 1
                if self.parada_temprana and self.epocas_sin_mejora >= self.parada_temprana:
                    print(f"Detenido en época {t+1}: sin mejora por {self.parada_temprana} épocas.")
                    break

        # Gráfica de pérdidas
        if plotear_perdida:
            plt.figure(figsize=(8, 6))
            plt.style.use('dark_background')
            plt.plot(range(1, len(self.perdida_entrenamiento)+1), self.perdida_entrenamiento, label='Train')
            if self.cargador_validacion:
                plt.plot(range(1, len(self.perdida_validacion)+1), self.perdida_validacion, label='Val')
            plt.title('Pérdida vs Época')
            plt.xlabel('Época')
            plt.ylabel('Pérdida')
            plt.legend()
            plt.show()

    def validar(self, imprimir_perdida=False) -> tuple[float, float]:
        self.modelo.eval()
        perdida_total = 0.0
        correctas = 0
        total = 0
        with torch.no_grad():
            for x_entrada, salida_esperada in self.cargador_validacion:
                x_entrada = x_entrada.to(self.device)
                salida_esperada = salida_esperada.to(self.device)

                salida = self.modelo(x_entrada)
                if self.usar_softmax:
                    salida = F.softmax(salida, dim=1)

                perdida = self.func_perdida_total(salida, salida_esperada)
                perdida_total += perdida.item()

                # Predicciones
                _, predicciones = torch.max(salida, dim=1)
                # Manejo de etiquetas one-hot vs índices
                if salida_esperada.ndim > 1 and salida_esperada.size(1) > 1:
                    _, verdaderas = torch.max(salida_esperada, dim=1)
                else:
                    verdaderas = salida_esperada

                correctas += (predicciones == verdaderas).sum().item()
                total += verdaderas.size(0)

        perdida_prom = perdida_total / len(self.cargador_validacion)
        precision = correctas / total if total > 0 else 0.0
        if imprimir_perdida:
            print(f"Pérdida val: {perdida_prom:.4f} — Precisión: {precision:.4f}")
        return perdida_prom, precision

    def guardar_checkpoint(self):
        self.mejor_modelo = self.modelo.state_dict().copy()

##########################################################################################################################################
##########################################################################################################################################
##########################################################################################################################################

class Evaluador:

  def __init__(self, modelo: torch.nn.Module, device: str = None, nombres_clases: list[str] = None):
    self.modelo = modelo
    self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
    self.nombres_clases = nombres_clases

  def evaluar(self, cargador_datos: torch.utils.data.DataLoader, input_shape: tuple = None):
    self.modelo.eval()
    self.modelo.to(self.device)

    todas_predicciones = []
    todas_verdaderas = []

    with torch.no_grad():
      for x, y in cargador_datos:
        if input_shape:
          x = x.view(-1, *input_shape)  # reshape explícito si se necesita
        x = x.to(self.device)
        y = y.to(self.device)

        salida = self.modelo(x)
        pred = torch.argmax(salida, dim=1)

        todas_predicciones.extend(pred.cpu().numpy())
        todas_verdaderas.extend(y.cpu().numpy())

    print("Reporte de clasificación:\n")
    print(classification_report(todas_verdaderas, todas_predicciones, digits=4, target_names=self.nombres_clases))

    matriz_conf = confusion_matrix(todas_verdaderas, todas_predicciones)
    etiquetas = self.nombres_clases if self.nombres_clases else [str(i) for i in range(len(matriz_conf))]

    plt.figure(figsize=(8, 6))
    sns.heatmap(matriz_conf, annot=True, fmt='d', cmap='Blues',
                xticklabels=etiquetas, yticklabels=etiquetas)
    plt.xlabel('Etiqueta predicha')
    plt.ylabel('Etiqueta verdadera')
    plt.title('Matriz de Confusión - Dataset de Prueba')
    plt.tight_layout()
    plt.show()

##########################################################################################################################################
##########################################################################################################################################
##########################################################################################################################################

class gestor ():

  def __init__(self, ruta_base_drive: str = "/content/drive/MyDrive/modelos_torch"):
    self.ruta_base = ruta_base_drive
    os.makedirs(self.ruta_base, exist_ok=True)

  def guardar_modelo(self, modelo: torch.nn.Module, nombre_archivo: str):
    ruta_completa = os.path.join(self.ruta_base, nombre_archivo)
    torch.save(modelo.state_dict(), ruta_completa)
    print(f"Modelo guardado en: {ruta_completa}")

  def cargar_modelo(self, modelo: torch.nn.Module, nombre_archivo: str, map_location=None):
    ruta_completa = os.path.join(self.ruta_base, nombre_archivo)
    if not os.path.exists(ruta_completa):
      raise FileNotFoundError(f"No se encontró el archivo: {ruta_completa}")

    modelo.load_state_dict(torch.load(ruta_completa, map_location=map_location))
    modelo.eval()  # Modo evaluación por defecto
    print(f"Modelo cargado desde: {ruta_completa}")
    return modelo

##########################################################################################################################################
##########################################################################################################################################
##########################################################################################################################################

class MiDataset (Dataset):

  ##################################
  def __init__ (self, ruta_archivo, normalizar = False):
    self.data = pd.read_csv(ruta_archivo)
    self.etiquetas = ['Iris-setosa','Iris-versicolor','Iris-virginica']

    self.X = self.data.iloc[:, :-1].values # esto los vuelve arrays numpy
    self.Y = self.data.iloc[:, -1].values

    if normalizar:
      self.normalizador = MinMaxScaler()
      self.X = self.normalizador.fit_transform(self.X)

            # aca hay 2 'hints', no afectan al codigo, son de ayuda al usuario
  #############################################
  def __getitem__ (self, indice: int) -> Tuple:
    x = torch.tensor(self.X[indice], dtype = torch.float32)
    y = torch.zeros(3, dtype=torch.float32)
    indice_etiqueta = self.etiquetas.index(self.Y[indice])
    y[indice_etiqueta] = 1.0
    return x, y

  ###################
  def __len__ (self):
    return self.data.shape[0]

  #########################################################################
  def particionar (self, ratio_e = 0.7, semilla = None, validacion = True):
    if ratio_e >= 1:
      raise ValueError(f'Ratio mal definido. Obervar valor.')
    indices = list(range(len(self)))
    i_entrenamiento, i_resto = train_test_split(
        indices, train_size = ratio_e, random_state = semilla)
    if validacion == True:
      i_validacion, i_prueba = train_test_split(
      i_resto, train_size = 0.5, random_state = semilla)
      datos_entrenamiento = Subset(self, i_entrenamiento)
      datos_validacion = Subset(self, i_validacion)
      datos_prueba = Subset(self, i_prueba)
      return datos_entrenamiento, datos_validacion, datos_prueba
    else:
      i_prueba = i_resto
      datos_entrenamiento = Subset(self, i_entrenamiento)
      datos_prueba = Subset(self, i_prueba)
      return datos_entrenamiento, datos_prueba

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
workdir = '/content/drive/MyDrive/Uni/Deep_Learning/Modelos'
os.makedirs(workdir, exist_ok = True)

ruta = '/content/drive/MyDrive/Uni/Deep_Learning/Iris.csv'
iris = MiDataset(ruta)
entrenamiento, validacion, prueba = iris.particionar(
    ratio_e = 0.7, validacion = True)

In [ ]:
for x, y in entrenamiento:
  print(x.shape, y.shape)
  break

In [ ]:
# para IRIS habia usado un MLP de forma n_entrada x 15 x n_salida con ReLu
n_entrada = entrenamiento[0][0].shape[0]
n_oculta = 15
n_salida = entrenamiento[0][1].shape[0]
#metodo_pesos = torch.nn.init.zeros_
metodo_pesos = torch.nn.init.xavier_normal_
#metodo_pesos = None
semilla = None

modelo_init_ceros = PerceptronMulticapa(n_entrada, n_oculta, n_salida, f_act = 'relu',
                                        metodo_init_pesos = metodo_pesos, semilla = semilla)

log_dir = f"runs/iris_{int(time.time())}"
writer = SummaryWriter(log_dir = log_dir)

lr = 0.01
optimizador = torch.optim.SGD(modelo_init_ceros.parameters(), lr = lr)
func_perdida = nn.MSELoss(reduction = 'mean')
epocas = 1000

entrenador = Entrenador(modelo_init_ceros, entrenamiento, optimizador, func_perdida, validacion)
entrenador.ajustar(epocas, imprimir_perdida = False,
                   plotear_perdida = False, escritor = writer)
writer.close() # esto podria (quiza?) estar dentro de la funcion ajustar

%load_ext tensorboard
%tensorboard --logdir runs

### **Ejercicio 2**: regularización $L_1$ y $L_2$ sobre $\textbf{w}$

Ahora vamos a agregar regularización sobre los pesos de la red, mediante un término extra en la función de pérdida. Notar que esto no es una propiedad del modelo en sí, por lo tanto no implica cambios en el `nn.Module`. Si usan un `Trainer`, pueden por ejemplo agregar esta opción como argumento al mismo.

a) Vamos a usar regularización $L_2$. Esto se puede hacer "manualmente" agregando el término en el bucle de entrenamiento, o agregando un `weight_decay` al optimizador (el valor que se le pasa a este argumento es simplemente el coeficiente $\lambda$ que multiplica al término $||\textbf{w}||_2^2$). Examinar el comportamiento de los pesos cuando se varía $\lambda$, desde valores chicos hasta valores grandes.

b) Ahora probemos con regularización $L_1$. Éste tipo de regularización se debe implementar manualmente ya que no viene como opción en el optimizador. Nuevamente, observar qué ocurre con los pesos de la red al variar el coeficiente de este término (usando Tensorboard).

Nuevamente, pueden repetir estos pasos para `NetCNN` de manera opcional.

In [ ]:
# todo queda igual que antes, salvo porque hay que especificar la regularizacion en el Entrenador

# para IRIS habia usado un MLP de forma n_entrada x 15 x n_salida con ReLu
n_entrada = entrenamiento[0][0].shape[0]
n_oculta = 15
n_salida = entrenamiento[0][1].shape[0]
metodo_pesos = torch.nn.init.xavier_normal_
semilla = None

modelo = PerceptronMulticapa(n_entrada, n_oculta, n_salida, f_act = 'relu',
                                        metodo_init_pesos = metodo_pesos, semilla = semilla)

log_dir = f"runs/iris_{int(time.time())}"
writer = SummaryWriter(log_dir = log_dir)

lr = 0.01
optimizador = torch.optim.SGD(modelo.parameters(), lr = lr)
func_perdida = nn.MSELoss(reduction = 'mean')
epocas = 1000

#regularizacion = None
regularizacion = (2, 0.01) ####### aca ####### se expresa (p, lambda)
entrenador = Entrenador(modelo, entrenamiento, optimizador, func_perdida, validacion, regularizador = regularizacion, parada_temprana = 50)
entrenador.ajustar(epocas, imprimir_perdida = False,
                   plotear_perdida = False, escritor = writer)
writer.close() # esto podria (quiza?) estar dentro de la funcion ajustar

%load_ext tensorboard
%tensorboard --logdir runs

Se ve en los histogramas de los pesos como van acercandose a cero conforme se dan las epocas.

En L1, se nota (quiza autoconfirmado) que algunos varios se vuelven cero, por la no disminucion del gradiente.

Algunas veces, el valor de los gradientes aumentaba con las epocas, cosa que no deberia si se acerca a los minimos.

Parece bastante facil caer en situaciones que disparan el early stopping en pocas epocas, algo aleatorio tambien.

### **Ejercicio 3**: normalización por lotes (_batch normalization_) y _dropout_.

_Nota para profes: el objetivo de las partes a y b de este ejercicio es que vean la diferencia entre aplicar BN a un MLP y a una CNN._ (`nn.BatchNorm1d(n_features)
` vs. `nn.BatchNorm2d(num_channels)`)

a) La normalización por lotes se realiza calculando estimadores de la media y la desviación estándar de las activaciones en cada lote. Sin embargo, cuando _usamos_ la red en modo evaluación no tenemos lotes necesariamente (podemos pasar de a un solo `data point` por ejemplo, en cuyo caso ni siquiera podemos estimar una desviación estándar). ¿Cómo se obtienen entonces estos valores?

b) Modificar `IrisMLP` para que incluya la posibilidad de usar normalización por lotes. Deben agregar un parámetro al constructor, `use_batch_norm`, que indica si se usa o no. (_ayuda: necesitarán `torch.nn.BatchNorm1d`_)

c) Modificar la `NetCNN` para que incluya normalización por lotes luego de cada capa convolucional. (_ayuda: van a necesitar `torch.nn.BatchNorm2d` ¿en qué difiere del anterior?_)

d) Agregar el uso de _dropout_, con una probabilidad que se pase como argumento al constructor (si ponemos cero, se debe recuperar el comportamiento original, sin _dropout_).

e) Entrenar ambas redes y comparar el desempeño. Recordar el uso de `model.train()` y `model.eval()`. ¿Qué efecto tiene cada uno? ¿Qué diferencia o relación existe entre usar el último y el gestor de contexto `with torch.no_grad()`?

In [ ]:
# a)
# Para casos en los que se evalua con batches de tamaño 1 (y de igual forma si es con batches mas grandes),
# se usan los promedios y varianzas acumulados durante la etapa de entrenamiento

# b)
# pruebo usar la modificacion de batch normalization en el MLP

ruta = '/content/drive/MyDrive/Uni/Deep_Learning/Iris.csv'
iris = MiDataset(ruta, True)
entrenamiento, validacion, prueba = iris.particionar(
    ratio_e = 0.7, validacion = True)

# aca necesito usar batches de tamaño diferente de 1, asi que:
lote_tam = 15
cargador_entrenamiento = DataLoader(entrenamiento, batch_size = lote_tam, shuffle = False)
cargador_validacion = DataLoader(validacion, batch_size = lote_tam, shuffle = False)

n_entrada = entrenamiento[0][0].shape[0]
n_oculta = 15
n_salida = entrenamiento[0][1].shape[0]
metodo_pesos = torch.nn.init.xavier_normal_
semilla = None

modelo = PerceptronMulticapa(n_entrada, n_oculta, n_salida, f_act = 'relu',
                             metodo_init_pesos = metodo_pesos, semilla = semilla,
                             batch_norm = True)

log_dir = f"runs/iris_{int(time.time())}"
writer = SummaryWriter(log_dir = log_dir)

lr = 0.01
optimizador = torch.optim.SGD(modelo.parameters(), lr = lr)
func_perdida = nn.MSELoss(reduction = 'mean')
epocas = 100

regularizacion = (2, 0.1) ####### aca ####### se expresa (p, lambda)
entrenador = Entrenador(modelo, cargador_entrenamiento, optimizador, func_perdida, cargador_validacion,
                        regularizador = regularizacion, parada_temprana = 50)
entrenador.ajustar(epocas, imprimir_perdida = False,
                   plotear_perdida = False, escritor = writer)
writer.close() # esto podria (quiza?) estar dentro de la funcion ajustar

%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# c
# ahora voy con la CNN

# cargo primero MNIST
import torchvision
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = [12, 10]
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
# for i in range(1,17):
#     plt.subplot(4,4,i)
#     plt.imshow(mnist_train.train_data[i,:,:], cmap=plt.get_cmap('gray_r'))

In [ ]:
f1 = 5
s1 = 1
p1 = 0
f2 = 5 # lo cambio como prueba de que pasa si reduzco el modelo
s2 = 1
p2 = 0
metodo_pesos = torch.nn.init.xavier_normal_
semilla = 17
dropout_val = 0.1

modelo = NetCNN(f1, s1, p1, f2, s2, p2, semilla = semilla,
                metodo_init_pesos = metodo_pesos, batch_norm = True,
                dropout_val = dropout_val)
              # recordar que en este modelo se usa relu predefinidamente

log_dir = f"runs/mnist_{int(time.time())}"
writer = SummaryWriter(log_dir = log_dir)

lr = 0.01
optimizador = torch.optim.SGD(modelo.parameters(), lr = lr)
func_perdida = nn.CrossEntropyLoss()
epocas = 5
regularizacion = (2, 0.1) ####### aca ####### se expresa (p, lambda)

lote_tam = 15
cargador_entrenamiento = DataLoader(mnist_train, batch_size = lote_tam, shuffle = True)
# quiero partir una parte de mnist_test para crear un cargador de validacion
validacion = torch.utils.data.Subset(mnist_test, range(len(mnist_test)//2))
cargador_validacion = DataLoader(validacion, batch_size = lote_tam, shuffle = False)

entrenador = Entrenador(modelo, cargador_entrenamiento, optimizador, func_perdida, cargador_validacion,
                        regularizador = regularizacion, parada_temprana = 50)
entrenador.ajustar(epocas, imprimir_perdida = False,
                   plotear_perdida = False, escritor = writer)
writer.close() # esto podria (quiza?) estar dentro de la funcion ajustar

%load_ext tensorboard
%tensorboard --logdir runs

La normalizacion en el caso de la red convolucional, se da entre todos los pixeles de cada uno de los features maps de un mismo canal para cada dato en un lote.

In [ ]:
ruta = '/content/drive/MyDrive/Uni/Deep_Learning/Iris.csv'
iris = MiDataset(ruta, True)
entrenamiento, validacion, prueba = iris.particionar(
    ratio_e = 0.7, validacion = True)

# aca necesito usar batches de tamaño diferente de 1, asi que:
lote_tam = 15
cargador_entrenamiento = DataLoader(entrenamiento, batch_size = lote_tam, shuffle = False)
cargador_validacion = DataLoader(validacion, batch_size = lote_tam, shuffle = False)

n_entrada = entrenamiento[0][0].shape[0]
n_oculta = 15
n_salida = entrenamiento[0][1].shape[0]
metodo_pesos = torch.nn.init.xavier_normal_
dropout_val = 0.1
semilla = 17

modelo = PerceptronMulticapa(n_entrada, n_oculta, n_salida, f_act = 'relu',
                             metodo_init_pesos = metodo_pesos, semilla = semilla,
                             batch_norm = True, dropout_val = dropout_val)

log_dir = f"runs/iris_{int(time.time())}"
writer = SummaryWriter(log_dir = log_dir)

lr = 0.01
optimizador = torch.optim.SGD(modelo.parameters(), lr = lr)
func_perdida = nn.MSELoss(reduction = 'mean')
epocas = 100

regularizacion = (2, 0.1) ####### aca ####### se expresa (p, lambda)
entrenador = Entrenador(modelo, cargador_entrenamiento, optimizador, func_perdida, cargador_validacion,
                        regularizador = regularizacion, parada_temprana = 50)
entrenador.ajustar(epocas, imprimir_perdida = False,
                   plotear_perdida = False, escritor = writer)
writer.close() # esto podria (quiza?) estar dentro de la funcion ajustar

%load_ext tensorboard
%tensorboard --logdir runs

|### **Ejercicio 4**: Optimizadores

En este ejercicio vamos a probar los distintos optimizadores vistos en la teoría: SGD, SGD con momento, Adagrad, Adam y AdamW:

```
optim.SGD(model.parameters(), lr=0.01)
optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optim.Adagrad(model.parameters(), lr=0.01)
optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
```

**Nota**: `Adam(..., weight_decay=0.01)` y `AdamW(..., weight_decay=0.01)` no hacen exactamente lo mismo, pueden encontrar una explicación en [este post](https://stackoverflow.com/questions/64621585/pytorch-optimizer-adamw-and-adam-with-weight-decay) por ejemplo.

a) En el problema de MNIST con `NetCNN`, lanzar $N$ entrenamientos con cada uno de estos optimizadores, promediar las curvas de la función de pérdida en función de las épocas y comparar la velocidad de convergencia y el desempeño final (pueden también graficar la _accuracy_ que es más fácil de interpretar).

b) Opcionalmente, repetir lo mismo con `IrisMLP`.

In [ ]:
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# Hiperparámetros fijos
f1, s1, p1 = 5, 1, 0
f2, s2, p2 = 5, 1, 0
metodo_pesos = torch.nn.init.xavier_normal_
semilla = 111
dropout_val = 0.1
batch_norm = True
lr = 0.01
epocas = 40
regularizacion = None
lote_tam = 15
func_perdida = nn.CrossEntropyLoss()

# DataLoaders
cargador_entrenamiento = DataLoader(mnist_train, batch_size=lote_tam, shuffle=True)
valid_split = Subset(mnist_test, list(range(len(mnist_test)//2)))
cargador_validacion = DataLoader(valid_split, batch_size=lote_tam, shuffle=False)

# Lista de optimizadores a probar
optimizadores = [
    lambda params: torch.optim.SGD(params, lr=0.01),
    lambda params: torch.optim.SGD(params, lr=0.01, momentum=0.9),
    lambda params: torch.optim.Adagrad(params, lr=0.01),
    lambda params: torch.optim.Adam(params, lr=0.001, weight_decay=0),
    lambda params: torch.optim.AdamW(params, lr=0.001, weight_decay=0.01),
]
nombres = [
    'SGD_lr0.01',
    'SGD_mom0.9',
    'Adagrad_lr0.01',
    'Adam_lr0.001',
    'AdamW_wd0.01',
]

for nombre, fabrica_opt in zip(nombres, optimizadores):
    print(f"\n=== Entrenando con: {nombre} ===")
    # Instancio modelo y optimizador
    modelo = NetCNN(
        f1, s1, p1, f2, s2, p2,
        semilla=semilla,
        metodo_init_pesos=metodo_pesos,
        batch_norm=batch_norm,
        dropout_val=dropout_val
    )
    optimizador = fabrica_opt(modelo.parameters())

    # TensorBoard
    log_dir = f"runs/mnist_{nombre}_{int(time.time())}"
    writer = SummaryWriter(log_dir=log_dir)

    # Entrenador
    entrenador = Entrenador(
        modelo=modelo,
        cargador_entrenamiento=cargador_entrenamiento,
        optimizador=optimizador,
        func_perdida=func_perdida,
        cargador_validacion=cargador_validacion,
        parada_temprana = 10,
        regularizador=regularizacion
    )
    entrenador.ajustar(
        epocas=epocas,
        imprimir_perdida=False,
        plotear_perdida=False,
        escritor=writer
    )
    writer.close()

# Una vez terminado, en un notebook:
%load_ext tensorboard
%tensorboard --logdir runs